<a href="https://colab.research.google.com/github/Kanyi254/Insurance-quotation/blob/Brandon/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

In [8]:
# Load the full datasets for Q3 and Q4
file_path_q4 = 'BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx'
file_path_q3 = 'Medical quota share treaty  3rd Qtr 2020.xlsx'

xls_q4 = pd.ExcelFile(file_path_q4)
xls_q3 = pd.ExcelFile(file_path_q3)

premium_q4_full = pd.read_excel(xls_q4, sheet_name='Premium Bordereaux')
claims_q4_full = pd.read_excel(xls_q4, sheet_name='Claims Bordereaux')
premium_q3_full = pd.read_excel(xls_q3, sheet_name='Premium Bordereaux')
claims_q3_full = pd.read_excel(xls_q3, sheet_name='Claims Bordereaux')

# Display column names to inspect
print("Premium Q4 Columns:", premium_q4_full.columns)
print("Claims Q4 Columns:", claims_q4_full.columns)
print("Premium Q3 Columns:", premium_q3_full.columns)
print("Claims Q3 Columns:", claims_q3_full.columns)

Premium Q4 Columns: Index(['Policy Holder ID', 'Principal beneficiary', 'Dependants',
       'Total beneficiaries', 'Police ID', 'Start Date of Cover',
       'End Date of Cover', ' ', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Premium Amount', 'Benefit Limit', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Premium Paid/Billed'],
      dtype='object')
Claims Q4 Columns: Index(['Policy Holder ID', 'Member ID', 'Start Date of Cover',
       'End Date of Cover', 'Unnamed: 4', 'Unnamed: 5',
       'Date of Claim/Treatment Date', 'Date of Payment/Approval Date',
       'Amount Claimed', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Amount Paid', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Benefit Limit', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Provider Name/Code'],
      dtype='object')
P

In [9]:
# Drop columns that are mostly empty or irrelevant
premium_q4_clean = premium_q4_full.dropna(axis=1, how='all')
claims_q4_clean = claims_q4_full.dropna(axis=1, how='all')
premium_q3_clean = premium_q3_full.dropna(axis=1, how='all')
claims_q3_clean = claims_q3_full.dropna(axis=1, how='all')

In [12]:
# Handle missing values by dropping rows with missing premium or claims data
# Replace with actual column names from your dataset
premium_q4_clean = premium_q4_clean.dropna(subset=['Premium Amount', 'Premium Paid/Billed']) # Replaced 'Premium', 'Paid to Date' with actual column names 'Premium Amount', 'Premium Paid/Billed'

# Changed 'Incurred to date' to 'Amount Claimed' based on column names provided
claims_q4_clean = claims_q4_clean.dropna(subset=['Amount Claimed', 'Policy Holder ID'])

premium_q3_clean = premium_q3_clean.dropna(subset=['Premium Amount', 'Premium Paid/Billed']) # Replaced 'Premium', 'Paid to Date' with actual column names 'Premium Amount', 'Premium Paid/Billed'
# Assuming 'Amount Claimed' is the equivalent for Q3 as well - adjust if needed
claims_q3_clean = claims_q3_clean.dropna(subset=['Amount Claimed', 'Policy Holder ID'])

# Display the cleaned data
print(premium_q4_clean.head())
print(claims_q4_clean.head())
print(premium_q3_clean.head())
print(claims_q3_clean.head())


               Policy Holder ID  Principal beneficiary Dependants  \
1           NDUWUMUKAMA CHARLES                    1.0          4   
2      NSABIYUMVA JEAN BERTRAND                    1.0          2   
3   BICOR VIE ET CAPITALISATION                   20.0         42   
4      NDABARUSHIMANA DIEUDONNE                    1.0          4   
14       BURUNDI LEND AND LEASE                    1.0          1   

    Total beneficiaries   Police ID  Start Date of Cover End Date of Cover  \
1                   5.0  20000030.0  2020-10-22 00:00:00        2021-10-21   
2                   3.0  19000002.0  2020-10-23 00:00:00        2021-10-22   
3                  62.0  19000005.0  2020-11-05 00:00:00        2021-11-04   
4                   5.0  20000031.0  2020-12-04 00:00:00        2021-12-03   
14                  2.0  20000012.0        November 2020        2021-05-04   

             Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11  Premium Amount  \
1     948885          1     948885    

In [15]:
from sklearn.ensemble import IsolationForest

# Assuming 'claims_q4_clean' and 'premium_q4_clean' are your cleaned datasets

# Step 1: Combine relevant features for anomaly detection
# Adjust these columns based on your dataset structure
# Replace with actual column names from your dataframe
# Use claims_q4_clean.columns to see the available column names
features = claims_q4_clean[['Amount Claimed']]

# Step 2: Train Isolation Forest model
iso_forest = IsolationForest(contamination=0.05)  # Adjust contamination based on desired sensitivity
iso_forest.fit(features)

# Step 3: Predict anomalies (-1 for anomaly, 1 for normal)
claims_q4_clean['anomaly'] = iso_forest.predict(features)

# Step 4: Filter out anomalies
anomalies = claims_q4_clean[claims_q4_clean['anomaly'] == -1]

# Display detected anomalies
print("Detected Anomalies:")
print(anomalies)

# You can also trigger alerts based on anomalies
if not anomalies.empty:
    print("Alert: Anomalies detected!")

Detected Anomalies:
                           Policy Holder ID           Member ID  \
2749  KFW BANQUE ALLEMANDE DE DEVELOPPEMENT  NZOBAMBONA ISIDORE   
2776               NSABIYUMVA JEAN BERTRAND      NKANZI ORNELLA   

     Start Date of Cover End Date of Cover Date of Claim/Treatment Date  \
2749          2020-01-01        2020-12-31          2020-10-06 00:00:00   
2776          2019-10-07        2020-10-06          2020-10-05 00:00:00   

     Date of Payment/Approval Date Amount Claimed Unnamed: 9 Unnamed: 10  \
2749           2020-10-06 00:00:00         127500        NaN         NaN   
2776           2020-10-05 00:00:00          87000        NaN         NaN   

     Unnamed: 11  ... Unnamed: 19 Unnamed: 20 Benefit Limit Unnamed: 22  \
2749         NaN  ...         NaN      127500       1500000    10000000   
2776         NaN  ...         NaN       87000       1200000     7200000   

     Unnamed: 23 Unnamed: 24 Unnamed: 25 Unnamed: 26 Provider Name/Code  \
2749      216000      

<ipython-input-15-70ed060b684f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_q4_clean['anomaly'] = iso_forest.predict(features)
